In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("Datasets/data_train.csv")

df[["user", "item"]] = df.Id.str.split("_", expand=True)

df.user = df.user.str.replace("r", "")
df.item = df.item.str.replace("c", "")

#########
df2 = pd.read_csv("Datasets/sample_submission.csv")

df2[["user", "item"]] = df2.Id.str.split("_", expand=True)

df2.user = df2.user.str.replace("r", "")
df2.item = df2.item.str.replace("c", "")

df.head()

,Id,Prediction,user,item
0,r44_c1,4,44,1
1,r61_c1,3,61,1
2,r67_c1,4,67,1
3,r72_c1,3,72,1
4,r86_c1,5,86,1


In [11]:
from surprise import Reader
from surprise import Dataset

reader = Reader(rating_scale=(1,5)) 
data = Dataset.load_from_df(df[["user","item","Prediction"]], reader)

In [12]:
from surprise import SVD # importer ici les algo qu'on testera
from surprise import model_selection

Dans la cell suivante, on va créer un "model selection" qui se base sur grid search cv. Ce dernier prend des paramètres et pourrait prendre plusieurs algos, a tester chacun selon les paramètres etc etc, mais ce n'est qu'une définition de model selection

In [13]:
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]} #ici il va falloir apprendre et définir les paramtères à tester

gs = model_selection.GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)  #définition du model selon les paramètres

C'est dans la celle suivante que se font actuellement les calculs de chaque algo défini au dessus, avec les paramètres etc
c'est normalement cette celle qui prendra mille an a tourner car elle calculera tout pour nous ressortir ce qu'il y a de mieux

In [14]:
gs.fit(data) # les calculs se font actuellement ici 

Principe de SKLearn et Surprise qui est basé sur le premier:
* définition d'un test model à travers model_selection.gridsearchcv
* testmodel.fit qui va tester les model et paramètres pour retourner ce qu'il y a de mieux
* algorithm = testmodel.best_estimator["type de erreur choisi"] nous donne le meilleur algo selon les meilleur paramtère
* algo.fit(data)
* algo.estimate(data a tester)

In [15]:
pd.DataFrame(gs.cv_results).head(2) #enlever head pour tout afficher

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,1.016604,1.019065,1.019457,1.018375,0.001263,4,0.836175,0.838323,0.837787,0.837428,0.000913,2,9.873740,0.218786,4.438072,0.101825,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,1.023789,1.026245,1.026634,1.025556,0.001260,7,0.843743,0.846079,0.845410,0.845078,0.000982,6,9.416066,0.167271,4.617349,0.831556,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6


In [16]:
print(gs.best_params )
print(gs.best_score)


{'rmse': {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}, 'mae': {'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}}
{'rmse': 1.0147312871491405, 'mae': 0.8334400451350668}


In [17]:
algo = gs.best_estimator["mae"] #choix de l'algo selon l'erreur, touuuut inclus

In [18]:
algo.fit(data.build_full_trainset()) # ici on va train notre algo sur le dataset complet, sans cv car les paramètres sont optimaux

La différence entre algo.predict et algo.estimate est que predict devrait être utilisé sur des valeurs que l'on connait d'avance, par exemple si on sait que le user 5 pour le film 12 a voté 3, on peut rentrer ces valeurs et voir "l'erreur"
algo.estimate fait littéralement ce qu'on veut pour le projet, c'est à dire estimer. Pas à 100% sur sur de ça ici mais c'est ce qui me parait le plus juste. Pour l'instant on boucle sur algo.estimate mais il y a peut etre mieux à faire 
il faudra round la valeur la remettre dans le pred.csv etc

In [19]:
algo.estimate(37,1) #premier à tester

4.175039848821002

Dessous, ce qu'on avait testé au début, on peut définir un algo ici SVD avec ses paramètres de bases, faire un model_selection.cross_validate dessus mais qui "ne sert a rien" car il faudrait comparer ça avec la même fonction mais pour d'autres algos
finalement on fit direct notre data, et on estimate

In [20]:
temp_algo = SVD()
model_selection.cross_validate(temp_algo, data, cv=2)

{'test_rmse': array([1.03829264, 1.03510797]),
 'test_mae': array([0.83187162, 0.82874481]),
 'fit_time': (27.694921255111694, 28.023682832717896),
 'test_time': (8.076480865478516, 5.6071672439575195)}

In [21]:
temp_algo.fit(data.build_full_trainset())

In [22]:
temp_algo.predict(37,1)

Prediction(uid=37, iid=1, r_ui=None, est=3.8572805008190647, details={'was_impossible': False})

In [23]:
temp_algo.estimate(37,1)

4.1280121789331625